In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.3 MB/s 
     |████████████████████████████████| 7.6 MB 2.8 MB/s 
     |████████████████████████████████| 182 kB 44.2 MB/s 


In [2]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [3]:
# example from https://towardsdatascience.com/clustering-the-20-newsgroups-dataset-with-gpt3-embeddings-10411a9ad150
import pandas as pd
from transformers import GPT2TokenizerFast
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train')
print(list(newsgroups_train.target_names))
print(newsgroups_train.data[:3])
print(newsgroups_train.target[:3])

df_news = pd.DataFrame(list(newsgroups_train.data), columns=['Text'])
df_news["Target"] = list(newsgroups_train.target)

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

df_news['n_tokens'] = df_news.Text.apply(lambda x: len(tokenizer.encode(x)))
df_news = df_news[df_news.n_tokens<2000]
print(len(df_news))

df_news.head(5)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
["From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever in

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1119 > 1024). Running this sequence through the model will result in indexing errors


10995


,Text,Target,n_tokens
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,211
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,238
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,506
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,271
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,326


In [4]:
import numpy as np

In [5]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('gpt2')  # or any other checkpoint
word_embeddings = model.transformer.wte.weight  # Word Token Embeddings 
position_embeddings = model.transformer.wpe.weight  # Word Position Embeddings

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [7]:
import pandas as pd
df = pd.read_csv("/drive/My Drive/Colab Notebooks/data/IST 736 projects/myslice/myslice-text clustering.csv")

In [8]:
docs = df[~df['myslice.dissatisfation.text'].isna()]
corpus = df[~df['myslice.dissatisfation.text'].isna()]

In [9]:
docs = docs['myslice.dissatisfation.text']
corpus = corpus['myslice.dissatisfation.text']

In [ ]:
#gpt_embeddings = word_embeddings.encode(corpus)

In [10]:
docs.head()

0    The tiles keep moving around and nothing is na...
1                                 Stale login request!
2    it seems like a hodgepodge and the items I use...
3                                                login
4          Poor navigation features for disabled users
Name: myslice.dissatisfation.text, dtype: object

In [11]:
crTokens = docs.apply(lambda x: len(tokenizer.encode(x)))
# df_news['n_tokens'] = df_news.Text.apply(lambda x: len(tokenizer.encode(x)))

In [12]:
crTokens

0     13
1      5
2     21
3      1
4      6
      ..
64     9
65     3
66     7
67    12
68     2
Name: myslice.dissatisfation.text, Length: 69, dtype: int64

In [13]:
cr = pd.DataFrame(docs, crTokens)

In [14]:
cr.head()

,myslice.dissatisfation.text
myslice.dissatisfation.text,
13,HR Functions (specifically Chartstring changes...
5,Locating tabs or links within its structure
21,Things listed multiple times
1,Stale login request!
6,Frequently will not load without clearing all ...


In [15]:
cr = pd.DataFrame(docs)

In [16]:
cr.head()

,myslice.dissatisfation.text
0,The tiles keep moving around and nothing is na...
1,Stale login request!
2,it seems like a hodgepodge and the items I use...
3,login
4,Poor navigation features for disabled users


In [17]:
cr = cr.merge(crTokens, left_index = True, right_index = True)

In [18]:
cr.head()

,myslice.dissatisfation.text_x,myslice.dissatisfation.text_y
0,The tiles keep moving around and nothing is na...,13
1,Stale login request!,5
2,it seems like a hodgepodge and the items I use...,21
3,login,1
4,Poor navigation features for disabled users,6


In [19]:
cr = cr.reset_index()

In [20]:
cr.head()

,index,myslice.dissatisfation.text_x,myslice.dissatisfation.text_y
0,0,The tiles keep moving around and nothing is na...,13
1,1,Stale login request!,5
2,2,it seems like a hodgepodge and the items I use...,21
3,3,login,1
4,4,Poor navigation features for disabled users,6


In [21]:
cr = cr.drop(columns = ['index'])
cr = cr.rename({"myslice.dissatisfation.text_x" : "myslice_dissatisfation_text"}, axis = 1)
cr = cr.rename({"myslice.dissatisfation.text_y" : "myslice_dissatisfation_token"}, axis = 1)

In [22]:
cr.head()

,myslice_dissatisfation_text,myslice_dissatisfation_token
0,The tiles keep moving around and nothing is na...,13
1,Stale login request!,5
2,it seems like a hodgepodge and the items I use...,21
3,login,1
4,Poor navigation features for disabled users,6


In [23]:
crTokens = crTokens.reset_index()

In [24]:
crTokens = crTokens.drop(columns = ['index'])

In [25]:
crVector = model.transformer.wte.weight[crTokens['myslice.dissatisfation.text']]

In [26]:
crTokens

,myslice.dissatisfation.text
0,13
1,5
2,21
3,1
4,6
...,...
64,9
65,3
66,7
67,12


In [27]:
crVector

tensor([[ 0.0466, -0.0113,  0.0283,  ..., -0.0735,  0.0496,  0.0963],
        [ 0.0112, -0.1511,  0.1904,  ..., -0.2641,  0.1187, -0.1229],
        [-0.0410, -0.0706,  0.1606,  ...,  0.0348,  0.1737, -0.0178],
        ...,
        [-0.1300, -0.2115,  0.1315,  ..., -0.0846, -0.2080, -0.0746],
        [ 0.0614, -0.0150,  0.0566,  ..., -0.0919, -0.0735, -0.0853],
        [-0.1275,  0.0479,  0.1841,  ...,  0.0899, -0.1297, -0.0879]],
       grad_fn=<IndexBackward0>)

In [28]:
crVectorDetach = crVector.detach()

In [29]:
crVectorDetach

tensor([[ 0.0466, -0.0113,  0.0283,  ..., -0.0735,  0.0496,  0.0963],
        [ 0.0112, -0.1511,  0.1904,  ..., -0.2641,  0.1187, -0.1229],
        [-0.0410, -0.0706,  0.1606,  ...,  0.0348,  0.1737, -0.0178],
        ...,
        [-0.1300, -0.2115,  0.1315,  ..., -0.0846, -0.2080, -0.0746],
        [ 0.0614, -0.0150,  0.0566,  ..., -0.0919, -0.0735, -0.0853],
        [-0.1275,  0.0479,  0.1841,  ...,  0.0899, -0.1297, -0.0879]])

In [30]:
from sklearn.cluster import KMeans
K =6  # change number of cluster if applicable
gpt_model = KMeans(n_clusters = K, random_state = 2022)
gpt_model.fit(crVectorDetach)
labels = gpt_model.labels_.tolist()  # Save cluster labels in a separate list

In [31]:
def print_docs_closest_to_centroids(model, vec, n):
  K = len(model.cluster_centers_)
  for j in range(K):
    d = model.transform(vec)[:, j] # transform all docs to cluster-distance space
    idx = np.argsort(d)[:n] # find n docs closest to centroid

    c_idx = [m for m, label in enumerate(model.labels_) if label==j]  # find the index of all docs in cluster j   
    print('\n\n======cluster #', j, ', cluster size:', len(c_idx))
    for i in idx:
      if i not in c_idx:
        print('[ this doc is in a different cluster #', model.labels_[i], '>>', corpus[i])
      else:
        print(corpus[i])
  return

In [32]:
def print_cluster_sizes(model):
  cnt_perCluster = {}
  for c in model.labels_:
    cnt_perCluster[c] = cnt_perCluster.get(c, 0) + 1
  print(cnt_perCluster)

In [33]:
print_cluster_sizes(gpt_model)

{2: 20, 5: 5, 0: 17, 1: 16, 3: 4, 4: 7}


In [34]:
print_docs_closest_to_centroids(gpt_model, crVectorDetach, 5)



======cluster # 0 , cluster size: 17
Degradation of menu system, loss of "breadcrumbs", extra clicks required to navigate
Constantly having to clear cache and told I am still logged into the platform when i am not.  
It lacks the opportunity to be customized for department specific needs and the layers to get permissions added on is tedious.
2FA; platform performance (spend a lot of time waiting on spinny wheels even on a fast computer)
difficult to find what you need - you have to drill down, and parking serivces is a mess!


======cluster # 1 , cluster size: 16
Compatibility with browsers
access to training 
Very clunky interface
Things listed multiple times
new interface missing functions


======cluster # 2 , cluster size: 20
The tiles keep moving around and nothing is named in an intuitive way
This platform looks outdated, and does not have strong UX features.
It is often hard to find what I'm looking for
Having to sign in/authenticate usage makes logging in cumbersome
many time

In [35]:
crVectorDetach.size()

torch.Size([69, 768])